In [1]:
import torch
import torch.nn as nn
import torchvision
from vggnet import VGGNet
from load_cifar10 import train_loader, test_loader
import os
import tensorboardX
from resnet import resnet
from mobilenetv1 import mobilenetv1_small
from inceptionModule import InceptionNetSmall
from pre_resnet import pytorch_resnet18

num_of_train 50000
num_of_test 10000


In [2]:
import _path
PATH_HEAD_DATA = _path.PATH_HEAD
PATH_LOG = r'{}_data_/data_log/'.format(PATH_HEAD_DATA)
PATH_LOG

'C:/Users/tianyunchuan/iCloudDrive/_data_/data_log/'

In [3]:
# if not os.path.exists('{}model'.format(PATH_LOG)):
#     os.mkdir('{}model'.format(PATH_LOG))

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
num_epoch = 20
batch_size = 100
lr = 0.01
# net = VGGNet().to(device)
# net = resnet().to(device)
# net = mobilenetv1_small().to(device)
# net = InceptionNetSmall().to(device)
net = pytorch_resnet18().to(device)
net


resnet18(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [7]:
# loss
loss_func = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

# lr
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [8]:
%%timeit

if not os.path.exists('{}log'.format(PATH_LOG)):
    os.mkdir('{}log'.format(PATH_LOG))
writer = tensorboardX.SummaryWriter('{}log'.format(PATH_LOG))
step_n = 0


for epoch in range(num_epoch):
    print('epoch is: ', epoch)  
    net.train()  # train BN dropout
    
    for i, data in enumerate(train_loader):
        net.train()  # train BN dropout
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)        

        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
        
        _, pred = torch.max(outputs, dim=1)
        correct = pred.eq(labels.data).cpu().sum()

#         print('train lr:', optimizer.state_dict()['param_groups'][0]['lr'])    
        print('epoch: ', epoch, 'train step: ', i, 'loss: ', loss.item(),\
             'mini-batch correct: ', 100.0 * correct.item() / batch_size,\
             'train lr:', optimizer.state_dict()['param_groups'][0]['lr'])
        
        writer.add_scalar('train loss', loss.item(), global_step=step_n)
        writer.add_scalar('train correct', 100.0 * correct.item() / batch_size, global_step=step_n)
        
        im = torchvision.utils.make_grid(inputs)
        writer.add_image('train im', im, global_step=step_n)
        
        step_n += 1
        
    if not os.path.exists('{}model'.format(PATH_LOG)):
        os.mkdir('{}model'.format(PATH_LOG))
    torch.save(net.state_dict(), '{}model/{}.pth'.format(PATH_LOG, epoch))
    scheduler.step()

    
    sum_loss = 0
    sum_correct = 0    
    for i, data in enumerate(test_loader):
        net.eval()  
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)        

        outputs = net(inputs)
        loss = loss_func(outputs, labels)        
        _, pred = torch.max(outputs, dim=1)
        correct = pred.eq(labels.data).cpu().sum()
        
        sum_loss += loss.item()
        sum_correct += correct.item()
        
        im = torchvision.utils.make_grid(inputs)
        writer.add_image('test im', im, global_step=step_n)
    
    test_loss = sum_loss * 1.0 / len(test_loader)
    test_correct = sum_correct * 100.0 / len(test_loader) / batch_size
    writer.add_scalar('test loss', loss.item(), global_step=epoch+1)
    writer.add_scalar('test correct', 100.0 * correct.item() / batch_size, global_step=epoch+1)
    print('epoch:', epoch, 'loss: ', test_loss,\
         'test correct: ', test_correct)
        
writer.close()

epoch is:  0
epoch:  0 train step:  0 loss:  2.6069536209106445 mini-batch correct:  9.0 train lr: 0.01
epoch:  0 train step:  1 loss:  5.928409099578857 mini-batch correct:  14.0 train lr: 0.01


KeyboardInterrupt: 